## Data Transform with Glue

이번 실습에서는 AWS Glue를 활용하여 데이터를 로딩, 변환, 저장하는 기본적인 데이터 처리와 관련된 코드를 확인하고 직접 수행해 봅니다.
이 노트북의 실행을 위해서는 Glue DevEndpoint 클러스터를 생성하고, SageMaker notebook 인스턴스를 연동한 환경 설정이 완료되어야 합니다. 

**[사전에 실습을 위한 데이터가 준비되지 않은 경우]**

AWS Glue에서 s3://analytics-data-seung/e-commerce 경로의 S3 데이터를 크롤링해서 analytics-source 라는 이름으로 데이터베이스를 생성하면 코드 실행이 가능합니다. 

기본적으로 데이터를 로딩하는 부분에서는 Glue의 API를 주로 사용하도록 합니다. Glue DynamicFrame에서는 대량의 파일을 로딩 / 적재 하는데 최적화된 API를 제공합니다.

데이터 로딩 이후 데이터의 변환에는 Spark DataFrame를 기본적으로 사용합니다.

### Jupyter Notebook 기본 사용법
* 명령줄 실행 : Ctrl + Enter
* 명령줄 추가 : 편집창이 아닌 부분 선택하고 A(위에 추가) or B(아래 추가)
* 정상적인 실행이 되지 않는 경우 Kernel 메뉴 - Restart Kernel 선택

### Glue API를 활용하여 데이터 로딩하기
아래 코드는 Glue에서 Job 생성시 기본 코드 템플릿에 포함된 부분입니다. 
노트북에서 Step by Step으로 진행하기 위해서 Job / Bookmark 관련된 일부 코드는 주석처리 하였습니다. 

먼저 AWS Glue의 주요 라이브러리를 로딩하고 Spark 작업을 실행하기 위한 GlueContext를 생성합니다.

In [1]:
#import sys
#from awsglue.transforms import *
#from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
#from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1558507935305_0010,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


Glue Data Catalog에 있는 정보를 기반으로 테이블 데이터를 로딩합니다. 다음 코드가 동작하기 위해서는 앞단계에서 Glue Crawler를 통해 Database와 테이블을 생성해주어야 합니다. 

order 테이블은 우리가 앞에서 생성한 analytics-source 데이터베이스에 포함된 테이블 입니다. 

이후 데이터 변환 작업의 편의성을 위해서 Glue의 DynamicFrame을 DataFrame 형식으로 변경합니다. 

In [2]:
order = glueContext.create_dynamic_frame.from_catalog(database="analytics-source", table_name="order").toDF()
print "Count: ", order.count()
order.printSchema()
order.show(5)

Count:  11283758
root
 |-- member_id: string (nullable = true)
 |-- order_date: long (nullable = true)
 |-- order_status: string (nullable = true)
 |-- country: string (nullable = true)
 |-- shipping_date: string (nullable = true)
 |-- total_price: long (nullable = true)
 |-- city: string (nullable = true)
 |-- order_time: long (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: long (nullable = true)
 |-- region: string (nullable = true)
 |-- order_id: string (nullable = true)

+---------+----------+------------+-------------+-------------+-----------+-------------+--------------+------------+-----------+------+--------------+
|member_id|order_date|order_status|      country|shipping_date|total_price|         city|    order_time|       state|postal_code|region|      order_id|
+---------+----------+------------+-------------+-------------+-----------+-------------+--------------+------------+-----------+------+--------------+
| ND-18370|  20161119|     shipped|Unit

이후 분석 단계에서 불필요한 컬럼을 미리 식별하여 데이터를 정리하는 작업을 진행합니다. 
몇몇 지역을 나타내는 컬럼의 데이터 분포를 살펴보고, 꼭 필요한 City 컬럼만 남기고 삭제하는 작업을 진행합니다. 

In [3]:
order.groupBy("country").count().sort("count", ascending=False).show(10)
order.groupBy("region").count().sort("count", ascending=False).show(10)
order.groupBy("state").count().sort("count", ascending=False).show(10)
order.groupBy("city").count().sort("count", ascending=False).show(10)
order.groupBy("postal_code").count().sort("count", ascending=False).show(10)

+-------------+--------+
|      country|   count|
+-------------+--------+
|United States|11283758|
+-------------+--------+

+-------+-------+
| region|  count|
+-------+-------+
|  South|3335405|
|   East|2819257|
|Central|2797985|
|   West|2331111|
+-------+-------+

+----------+-------+
|     state|  count|
+----------+-------+
|   Florida|1409972|
|  New York|1365079|
|      Ohio| 983351|
|  Virginia| 940477|
|California| 896780|
|     Texas| 896405|
|  Illinois| 895355|
|  Kentucky| 515686|
|    Kansas| 514542|
|  Colorado| 492511|
+----------+-------+
only showing top 10 rows

+-------------+------+
|         city| count|
+-------------+------+
|San Francisco|896024|
|     Elmhurst|895081|
|New York City|894701|
|    Henderson|515644|
|  Garden City|514531|
|       Denver|492449|
|   Chesapeake|491955|
|     Columbus|491824|
|       Toledo|491417|
|      Detroit|491338|
+-------------+------+
only showing top 10 rows

+-----------+------+
|postal_code| count|
+-----------+------

지역을 나타내는 컬럼인 country, region, state 컬럼 보다 주로 city 관련 컬럼의 활용도가 높다고 판단되고, postal_code는 city와 중복되므로, country, region, state, postal_code 컬럼은 삭제합니다. 

In [4]:
order.printSchema()
order_df = order.drop("country", "region", "state", "postal_code")
order_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- order_date: long (nullable = true)
 |-- order_status: string (nullable = true)
 |-- country: string (nullable = true)
 |-- shipping_date: string (nullable = true)
 |-- total_price: long (nullable = true)
 |-- city: string (nullable = true)
 |-- order_time: long (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: long (nullable = true)
 |-- region: string (nullable = true)
 |-- order_id: string (nullable = true)

root
 |-- member_id: string (nullable = true)
 |-- order_date: long (nullable = true)
 |-- order_status: string (nullable = true)
 |-- shipping_date: string (nullable = true)
 |-- total_price: long (nullable = true)
 |-- city: string (nullable = true)
 |-- order_time: long (nullable = true)
 |-- order_id: string (nullable = true)

회원별 주문 현황을 살펴보기 위해서 member 테이블을 추가로 로딩하여 order 테이블과 조인하여 member_order 테이블을 생성합니다.

In [5]:
member = glueContext.create_dynamic_frame.from_catalog(database="analytics-source", table_name="member").toDF()
member_order = member.join(order_df, "member_id")
member_order.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- login_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- city: string (nullable = true)
 |-- last_login_ymdt: long (nullable = true)
 |-- membership_level: string (nullable = true)
 |-- login_password: string (nullable = true)
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: long (nullable = true)
 |-- region: string (nullable = true)
 |-- age: long (nullable = true)
 |-- reg_ymdt: long (nullable = true)
 |-- order_date: long (nullable = true)
 |-- order_status: string (nullable = true)
 |-- shipping_date: string (nullable = true)
 |-- total_price: long (nullable = true)
 |-- city: string (nullable = true)
 |-- order_time: long (nullable = true)
 |-- order_id: string (nullable = true)

회원별 구매 횟수 / 구매 금액 / 나이별, 성별별 구매 금액 등을 계산해서 확인해봅니다. 

In [6]:
member_order_total = member_order.groupBy("member_id").count().sort("count", ascending=False)
member_order_total.show(10)

+---------+------+
|member_id| count|
+---------+------+
| AA-10315|895764|
| VF-21715|895084|
| SB-20290|515632|
| BT-11395|514534|
| KT-16480|492442|
| CK-12325|491950|
| RC-19960|491719|
| JS-15880|491407|
| HW-14935|491293|
| JH-15910|490843|
+---------+------+
only showing top 10 rows

In [7]:
member_order.filter(member_order.age.between(20, 29)).groupBy("age").sum("total_price").sort("sum(total_price)", ascending=False).show()

+---+----------------+
|age|sum(total_price)|
+---+----------------+
| 20|       117953446|
| 21|       117859681|
| 24|       117808273|
| 22|           75709|
| 23|           53856|
| 28|           46637|
| 27|           45551|
| 29|           38564|
| 26|           37624|
| 25|           21135|
+---+----------------+

In [8]:
member_order.groupBy("age").pivot("gender").sum("total_price").show(20)

+---+---------+---------+
|age|        F|        M|
+---+---------+---------+
| 29|    24275|    14289|
| 26|    27259|    10365|
| 54|    15415|    26019|
| 22|    26428|    49281|
| 34|128960410|135473362|
| 50|    23163|234943922|
| 57|123765352|    26313|
| 32|129217476|    15627|
| 43|    27115|    47471|
| 31|    22604|118099004|
| 39|    17217|123263172|
| 25|    11946|     9189|
| 58|    21880|129192137|
| 63|129016433|128557762|
| 27|    31955|    13596|
| 56|    19263|123665468|
| 51|    13105|    27175|
| 52|    40097|    19181|
| 41|    16181|    23351|
| 28|    13664|    32973|
+---+---------+---------+
only showing top 20 rows

이후 분석에서 주문별 상세 주문 내역을 활용하기 위해서 필요한 모든 테이블을 로딩하여, order 테이블과 조인하여 order_detali 테이블을 만듭니다. 

In [9]:
order_item = glueContext.create_dynamic_frame.from_catalog(database="analytics-source", table_name="order_item").toDF()
print "Count: ", order_item.count()
order_item.printSchema()

Count:  22573388
root
 |-- item_count: long (nullable = true)
 |-- order_date: long (nullable = true)
 |-- item_id: string (nullable = true)
 |-- item_price: long (nullable = true)
 |-- order_time: long (nullable = true)
 |-- order_id: string (nullable = true)

In [10]:
item = glueContext.create_dynamic_frame.from_catalog(database="analytics-source", table_name="item").toDF()
item_category = glueContext.create_dynamic_frame.from_catalog(database="analytics-source", table_name="item_category").toDF()

In [23]:
order_detail = order.join(order_item.join(item.join(item_category, item.item_category_id == item_category.category_id), "item_id"), "order_id")
order_detail.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- order_date: long (nullable = true)
 |-- order_status: string (nullable = true)
 |-- country: string (nullable = true)
 |-- shipping_date: string (nullable = true)
 |-- total_price: long (nullable = true)
 |-- city: string (nullable = true)
 |-- order_time: long (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: long (nullable = true)
 |-- region: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- item_count: long (nullable = true)
 |-- order_date: long (nullable = true)
 |-- item_price: long (nullable = true)
 |-- order_time: long (nullable = true)
 |-- price: long (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- reg_ymdt: long (nullable = true)
 |-- item_category_id: long (nullable = true)
 |-- category_name: string (nullable = true)
 |-- category_id: long (nullable = true)
 |-- parent_category_name: str

### Glue DynamicFrame을 활용하여 S3에 데이터 저장하기
앞쪽에서 Dataframe을 통해서 데이터 정리 작업과 Join을 완료한 데이터 파일을 별도의 S3 버킷에 저장합니다.
(DataFrame으로 변경한 데이터는 DynamicFrame로 변경하는 작업이 추가됩니다.)
저장된 데이터를 기반으로 ad-hoc 쿼리와 분석을 수행하도록 합니다. 

In [25]:
from awsglue.dynamicframe import DynamicFrame

order_detail_dyf = DynamicFrame.fromDF(order_detail, glueContext, 'order_detail_dyf')
member_order_total = DynamicFrame.fromDF(member_order, glueContext, 'member_order_total')

데이터를 저장할 S3 버킷명을 지정합니다. 앞 단계에서 생성한 각자 아이디를 버킷 이름에 넣어줍니다. 

In [13]:
#s3_bucket = 's3://analytics-data-[자신의 ID]' 
s3_bucket = 's3://analytics-data-seung' 

order_detail 테이블은 전체 데이터를 조인하여 분석용 테이블로 통합한 파일이므로, 이후 쿼리 성능을 위해서 날짜 컬럼인 order_date로 파티션을 해서 저장합니다. 또한 다른 테이블과 별도로 관리하기 위해서 별도의 S3 경로에 저장합니다. 

In [26]:
datasink0 = glueContext.write_dynamic_frame.from_options(frame = order_detail_dyf, connection_type = "s3", connection_options = {"path": s3_bucket + '/e-commerce-analytics_transformed/order_detail', "partitionKeys": "order_date"}, format = "parquet", transformation_ctx = "datasink0")  


In [17]:
datasink1 = glueContext.write_dynamic_frame.from_options(frame = member_order_total, connection_type = "s3", connection_options = {"path": s3_bucket + '/e-commerce-analytics/member_order'}, format = "parquet", transformation_ctx = "datasink1")


추가적으로 필요한 데이터를 모두 로딩해서 저장하도록 합니다. 
order (이전에 로딩 완료)
order_item (이전에 로딩 완료)
member
item
item_category

앞에서 로딩한여 DataFrame으로 변경하여 사용했던 테이블은 다시 DynamicFrame으로 변경하여 저장합니다.

In [18]:
order_dyf = DynamicFrame.fromDF(order, glueContext, 'order_dyf')
order_item_dyf = DynamicFrame.fromDF(order_item, glueContext, 'order_item_dyf')
member_dyf = DynamicFrame.fromDF(member, glueContext, 'member_dyf')
item_dyf = DynamicFrame.fromDF(item, glueContext, 'item_dyf')
item_category_dyf = DynamicFrame.fromDF(item_category, glueContext, 'item_category_dyf')

In [19]:
datasink2 = glueContext.write_dynamic_frame.from_options(frame = order_dyf, connection_type = "s3", connection_options = {"path": s3_bucket + '/e-commerce-analytics/order'}, format = "parquet", transformation_ctx = "datasink2")
datasink3 = glueContext.write_dynamic_frame.from_options(frame = order_item_dyf, connection_type = "s3", connection_options = {"path": s3_bucket + '/e-commerce-analytics/order_item'}, format = "parquet", transformation_ctx = "datasink3")
datasink4 = glueContext.write_dynamic_frame.from_options(frame = member_dyf, connection_type = "s3", connection_options = {"path": s3_bucket + '/e-commerce-analytics/member'}, format = "parquet", transformation_ctx = "datasink4")
datasink5 = glueContext.write_dynamic_frame.from_options(frame = item_dyf, connection_type = "s3", connection_options = {"path": s3_bucket + '/e-commerce-analytics/item'}, format = "parquet", transformation_ctx = "datasink5")
datasink6 = glueContext.write_dynamic_frame.from_options(frame = item_category_dyf, connection_type = "s3", connection_options = {"path": s3_bucket + '/e-commerce-analytics/item_category'}, format = "parquet", transformation_ctx = "datasink6")

여기까지 완료하면 우리가 분석에 사용할 e-commerce 관련된 OLTP 성격의 데이터를 모두 S3에 저장하였고, 또한 데이터 탐색과 변환을 통해 만들어진 몇몇 테이블도 S3에 저장하였습니다.
이후 다시 Lab을 따라서 다음 단계를 진행합니다. 